In [1]:
# Read in the csv file as a dataframe using pandas
import pandas as pd
df = pd.read_csv("quiz4_invest.csv", index_col='Project')

# read the parameters
Capital = {}
Sector = {}
Risk = {}
Duration = {}
ExpectedReturn = {}

for i in df.index:  # iterate over the row index
    Capital[i] = float(df['Capital ($M)'][i])
    Sector[i] = df['Sector'][i]
    Risk[i] = df['Risk'][i]
    Duration[i] = float(df['Duration'][i])
    ExpectedReturn[i] = float(df['Expected return ($M)'][i])
    
Projects = Capital.keys()
Budget = 100

In [2]:
print(df)

            Capital ($M)       Sector    Risk  Duration  Expected return ($M)
Project                                                                      
Project 1             20           IT     Low         2                    34
Project 2             15   Healthcare  Medium         4                    36
Project 3             23   Healthcare    High         3                    50
Project 4             10  Industrials  Medium         3                    16
Project 5             20  Engineering  Medium         6                    54
Project 6             12           IT    High         4                    32
Project 7             20   Financials    High         5                    60
Project 8             25       Energy     Low         7                    48
Project 9             14       Energy  Medium         4                    38
Project 10            10           IT     Low         1                    12


In [3]:
from docplex.mp.model import Model
mdl = Model()

In [4]:
# variables
select = mdl.binary_var_dict(Projects, name='select')

In [5]:
# objective
mdl.maximize(mdl.sum(ExpectedReturn[i] * select[i] for i in Projects))

In [6]:
# Constraints

# No more than the budget can be invested in total
mdl.add_constraint(mdl.sum(Capital[i] * select[i] for i in Projects) <= Budget)

# At most 2 projects in IT can be selected
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Sector[i] == 'IT') <= 2)

# At least one healthcare project must be selected
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Sector[i] == 'Healthcare') >= 1)

# The number of projects with Low risk must be at least the number of projects with High risk
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Risk[i] == 'Low') >= mdl.sum(select[i] for i in Projects if Risk[i] == 'High'))

# At least three projects with a duration of 4 years or shorter must be selected
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Duration[i] <= 4) >= 3)

docplex.mp.LinearConstraint[](select_Project 1+select_Project 2+select_Project 3+select_Project 4+select_Project 6+select_Project 9+select_Project 10,GE,3)

In [7]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.016,status='integer optimal solution')

In [8]:
mdl.print_solution()

objective: 238.000
status: OPTIMAL_SOLUTION(2)
  "select_Project 1"=1
  "select_Project 2"=1
  "select_Project 4"=1
  "select_Project 5"=1
  "select_Project 7"=1
  "select_Project 9"=1
